In [7]:
%load_ext autoreload
%autoreload 2

import time
import cv2
import numpy as np
import imutils
import pandas as pd
import mediapipe as mp
from enum import Enum

from src.hands_finder import HandsFinder
from src.video_utils import VideoCapture, add_texts, display_warning
import src.video_utils as vc_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
data_file_out = "out/gestures.csv"

def get_init_gestures():
    a_letter = ord('a')
    gestures = ['ok', 'victory', 'call', 'hang_in', 'one_up', 'two_up', 'hand_closed', 'hand_open', 'machedici', 'random']
    gestures = {chr(a_letter + x): [gestures[x], 0] for x in range(len(gestures))}
    return gestures

get_init_gestures()

{'a': ['ok', 0],
 'b': ['victory', 0],
 'c': ['call', 0],
 'd': ['hang_in', 0],
 'e': ['one_up', 0],
 'f': ['two_up', 0],
 'g': ['hand_closed', 0],
 'h': ['hand_open', 0],
 'i': ['machedici', 0],
 'j': ['random', 0]}

## Record Gestures

In [18]:
detector = HandsFinder()
vc = VideoCapture(fps=8)

gestures = get_init_gestures()
results = []


class HandStatus(Enum):
    K_HAND_ERROR = 1
    K_HAND_ADDED = 2
    NO_KEY = 3


def get_specs(status):

    if status == HandStatus.NO_KEY:
        return drawer.DrawingSpec(
            color=(255, 0, 255), thickness=5, circle_radius=5
        )

    if status == HandStatus.K_HAND_ERROR:
        return drawer.DrawingSpec(
            color=(0,0,255), thickness=5, circle_radius=5
        )

    if status == HandStatus.K_HAND_ADDED:
        return drawer.DrawingSpec(
            color=(0, 128, 0), thickness=5, circle_radius=5
        )

    return None

while vc.take_next():

    img = vc.get_frame()

    detector.fit(img)

    status = None

    if detector.found_hands():

        if vc.get_key() is not None:

            lm_norms = detector.get_positions_normalized()

            for h_k, lm_norm in enumerate(lm_norms):
                # mediapipe tries to guess hands parts if they are not in the
                # screen, let's not take those
                if detector.hand_fits_screen(img, h_k):
                    try:
                        gestures[vc.get_key()][1] += 1
                        results.append([gestures[vc.get_key() ][0], lm_norm])
                        status = HandStatus.K_HAND_ADDED
                    except KeyError:
                        img = display_warning(img)
                        status = HandStatus.K_HAND_ERROR

                else:
                    img = display_warning(img)
                    status = HandStatus.K_HAND_ERROR

        else:
            status = HandStatus.NO_KEY

        draw_spec = get_specs(status)

        img = detector.transform_connect_lines(img, draw_spec)

    img = cv2.flip(img, 1)
    text = [f"[{x[0]}]-{x[1][0]}: {x[1][1]}: " for x in gestures.items()]
    add_texts(img, text)
    cv2.imshow("Image", img)


vc.finish()


## Save in file

In [5]:
target = [r[0] for r in results]
array = [r[1] for r in results]


df = pd.DataFrame(array)
df = df.round(4)
cols = df.columns
df['target'] = target

# Target should be at the beginning
cols = cols.insert(0, 'target')

df[cols].to_csv(data_file_out, index=False, header=False, mode='a')